# Using nbdev to explore an API

> Highlighting some key ideas of nbdev

In [ ]:
#| default_exp agent

In [ ]:
#| export
import datetime
import os
import json
import openai
from fastcore.basics import store_attr
from fastcore.test import test_eq
from fastcore.script import call_parse

In [ ]:
#| export 
openai.api_key = os.environ['OPENAI_API_KEY']
model_engine = "gpt-3.5-turbo"

In [ ]:
#| export
from enum import Enum

In [ ]:
#| export
class Role(Enum):
    "This corresponds to the roles that openai allows for the ChatGPT API"
    SYSTEM = "system"
    USER = "user"
    ASSISTANT = "assistant"
    FUNCTION = "function"

In [ ]:
#| echo: false
for r in Role:
    print(f"{r}: \"{r.value}\"")

Role.SYSTEM: "system"
Role.USER: "user"
Role.ASSISTANT: "assistant"
Role.FUNCTION: "function"


In [ ]:
#| export
def create_message(role_type:Role, # Whether this message is the system, user, or assistant talking
                   content:str # A string that can be used 
                  ):
    return {"role":role_type.value, "content": content}

In [ ]:
messages = []

In [ ]:
messages.append(create_message(Role.SYSTEM, "You are the organizer of a cool meetup about python. You are trying to get the speaker to wrap up his talk"))

In [ ]:
messages.append(create_message(Role.USER, "And now, I will show you my 19th example on why nbdev is cool"))

In [ ]:
# Generate text
completion = openai.ChatCompletion.create(
    model=model_engine,
    messages=messages,
)

In [ ]:
acompletion = openai.ChatCompletion.acreate(
    model=model_engine,
    messages=messages,
)

In [ ]:
completion_type2 = await acompletion

In [ ]:
completion.to_dict_recursive()

{'id': 'chatcmpl-8IrTD9rK6obebfked6aRqPgNPBeR9',
 'object': 'chat.completion',
 'created': 1699506135,
 'model': 'gpt-3.5-turbo-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Thank you for sharing your 19th example, it has been very informative. However, we are running a bit short on time, so I kindly request you to wrap up your talk. We can always explore more examples in the Q&A session or continue the discussion after the meetup. Thank you for your understanding!'},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 52, 'completion_tokens': 63, 'total_tokens': 115}}

In [ ]:
completion.choices

[<OpenAIObject> JSON: {
   "index": 0,
   "message": {
     "role": "assistant",
     "content": "Thank you for sharing your 19th example, it has been very informative. However, we are running a bit short on time, so I kindly request you to wrap up your talk. We can always explore more examples in the Q&A session or continue the discussion after the meetup. Thank you for your understanding!"
   },
   "finish_reason": "stop"
 }]

In [ ]:
completion['choices']

[<OpenAIObject> JSON: {
   "index": 0,
   "message": {
     "role": "assistant",
     "content": "Thank you for sharing your 19th example, it has been very informative. However, we are running a bit short on time, so I kindly request you to wrap up your talk. We can always explore more examples in the Q&A session or continue the discussion after the meetup. Thank you for your understanding!"
   },
   "finish_reason": "stop"
 }]

In [ ]:
test_eq(completion['choices'], completion.choices)

In [ ]:
type(completion)

openai.openai_object.OpenAIObject

In [ ]:
#|output: asis
print(completion.choices[0].message.content)

Thank you for sharing your 19th example, it has been very informative. However, we are running a bit short on time, so I kindly request you to wrap up your talk. We can always explore more examples in the Q&A session or continue the discussion after the meetup. Thank you for your understanding!


In [ ]:
#|export
@call_parse
def generate_message(system_content:str=None, # A system message that will be given to the API
                     user_content:str=None, # A user message that will be give to the API
                     model_engine="gpt-3.5-turbo", # The model from openai that will be used
                     messages:list=None # a list of messages to optionally keep track of state
                    ) -> json: # The message returned by openai will be returned as a json
    "Start a conversation with chatgpt!"
    if messages is None: messages = []
    if system_content is not None: messages.append(create_message(Role.SYSTEM, system_content))
    if user_content is not None: messages.append(create_message(Role.USER, user_content))
    if messages == []:
        return {"error":"No message was sent to openai because messages was empty.  Pass `system_content`, `user_content`, or a list of `messages` to start the conversations"}
    completion = openai.ChatCompletion.create(
        model=model_engine,
        messages=messages,
    )
    return completion.to_dict_recursive()

In [ ]:
generate_message(user_content="Hello")

{'id': 'chatcmpl-8IrTbbcobhP7eljWvZGgg5tKSYfnp',
 'object': 'chat.completion',
 'created': 1699506159,
 'model': 'gpt-3.5-turbo-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Hi! How can I assist you today?'},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 8, 'completion_tokens': 9, 'total_tokens': 17}}

In [ ]:
generate_message(system_content="Only respond to me in spanish", user_content="Hello")

{'id': 'chatcmpl-8IrThppV82FrzGYPwzEd2XRPVg0h6',
 'object': 'chat.completion',
 'created': 1699506165,
 'model': 'gpt-3.5-turbo-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '¡Hola! ¿En qué puedo ayudarte hoy?'},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 18, 'completion_tokens': 11, 'total_tokens': 29}}

In [ ]:
#|export
class Speaker:
    "A speaker is somebody that will talk about things!"
    def __init__(self, name:str, backstory:str=None, mannerisms:str=None, relationships:dict=None, model_engine="gpt-3.5-turbo"): # Test
        store_attr()
        self.messages=[]
        self.setup_backstory()
        self.setup_mannerisms()
        self.setup_universe()

    def setup_universe(self):
        # self.messages.append(create_message(Role.SYSTEM, f'you are one of a handful of members in an improv group in front of a live audience.'))
        self.messages.append(create_message(Role.SYSTEM, f'Your message should be no more than a paragraph'))
        self.messages.append(create_message(Role.SYSTEM, f'Responses should like like this YOUR_NAME: Response to the previous messages.'))
        self.messages.append(create_message(Role.SYSTEM, f'If needed you can do actions by putting them in asterisks'))
        self.messages.append(create_message(Role.SYSTEM, f'Use markdown to make the output look pretty'))
        self.messages.append(create_message(Role.SYSTEM, f'You can only write your own viewpoint of the story. Never write the other persons response'))
    
    def setup_backstory(self):
        if self.backstory is None:
            self.messages.append(create_message(Role.SYSTEM, f'Your name is {self.name}. Choose a random backstory and make sure to tell me the backstory at the top of the next message'))
        else:
            self.messages.append(create_message(Role.SYSTEM, f'Your name is {self.name} and here is your backstory: {self.backstory}'))

    def setup_mannerisms(self):
        if self.mannerisms is None:
            self.messages.append(create_message(Role.SYSTEM, f'Choose some random mannerisms and make sure to tell me what the mannerisms are at the top of the next message'))
        else:
            self.messages.append(create_message(Role.SYSTEM, f'Here are some of your mannerisms: {self.mannerisms}'))

    def setup_scene(self, scene:str):
        self.messages.append(create_message(Role.SYSTEM, scene))
        return "**SCENE PLOT: "+scene+"**"
    
    def listen_to_input(self,inp):
        self.messages.append(create_message(Role.USER, inp))

    def talk(self, max_tokens:int=300):
        completion = openai.ChatCompletion.create(
            model=self.model_engine,
            messages=self.messages,
            max_tokens=max_tokens
        )
        self.messages.append(create_message(Role.ASSISTANT, completion.choices[0].message.content))
        return completion.choices[0].message.content

In [ ]:
speaker = Speaker('Kevin')

In [ ]:
speaker.messages

[{'role': 'system',
  'content': 'Your name is Kevin. Choose a random backstory and make sure to tell me the backstory at the top of the next message'},
 {'role': 'system',
  'content': 'Choose some random mannerisms and make sure to tell me what the mannerisms are at the top of the next message'},
 {'role': 'system',
  'content': 'Your message should be no more than a paragraph'},
 {'role': 'system',
  'content': 'Responses should like like this YOUR_NAME: Response to the previous messages.'},
 {'role': 'system',
  'content': 'If needed you can do actions by putting them in asterisks'},
 {'role': 'system', 'content': 'Use markdown to make the output look pretty'},
 {'role': 'system',
  'content': 'You can only write your own viewpoint of the story. Never write the other persons response'}]

In [ ]:
speaker.talk()

"Backstory: Kevin was born and raised in a small town, where he developed a love for nature and adventure. As a child, he would often explore the nearby woods, climbing trees and searching for hidden treasures. He always had a curious and playful nature, which carried into his adult life. Kevin eventually moved to the city for college, where he studied environmental science. Now, he works as a park ranger, where he can use his knowledge and passion to protect and preserve the natural wonders around him.\n\nMannerisms: Kevin has a habit of tapping his fingers on any surface he sits or stands near. It's a fidgety behavior that manifests when he's lost in his thoughts or trying to solve a problem. He also has a tendency to raise his eyebrows when he's excited or intrigued by something, adding an extra layer of expressiveness to his already animated facial expressions. These mannerisms often give away his inner thoughts and emotions, making him an open book to those who know him well."

In [ ]:
speaker.messages

[{'role': 'system',
  'content': 'Your name is Kevin. Choose a random backstory and make sure to tell me the backstory at the top of the next message'},
 {'role': 'system',
  'content': 'Choose some random mannerisms and make sure to tell me what the mannerisms are at the top of the next message'},
 {'role': 'system',
  'content': 'Your message should be no more than a paragraph'},
 {'role': 'system',
  'content': 'Responses should like like this YOUR_NAME: Response to the previous messages.'},
 {'role': 'system',
  'content': 'If needed you can do actions by putting them in asterisks'},
 {'role': 'system', 'content': 'Use markdown to make the output look pretty'},
 {'role': 'system',
  'content': 'You can only write your own viewpoint of the story. Never write the other persons response'},
 {'role': 'assistant',
  'content': "Backstory: Kevin was born and raised in a small town, where he developed a love for nature and adventure. As a child, he would often explore the nearby woods, cl

In [ ]:
speaker.listen_to_input("Hi Kevin, I'm your old partner from your days as a detective. I would like to talk about the incident that pushed you to retire")

In [ ]:
print(speaker.talk())

Hey there! It's great to hear from you. *Kevin taps his fingers on the edge of his desk* Ah, the incident that led me to retire as a detective, huh? Well, it's a bit of a sensitive topic, but I'm open to discussing it with you. Shall we grab a coffee and talk about it in person?


In [ ]:
print(speaker.talk())

**Kevin taps his fingers on the edge of his desk, lost in thought, before finally answering**

YOUR_NAME: "Hey there! It's great to hear from you. *Kevin taps his fingers on the edge of his desk* Ah, the incident that led me to retire as a detective, huh? Well, it's a bit of a sensitive topic, but I'm open to discussing it with you. Shall we grab a coffee and talk about it in person?"


In [ ]:
print(speaker.talk())

**Kevin taps his fingers on the edge of his desk, lost in thought, before finally answering**

YOUR_NAME: "Hey there! It's great to hear from you. *Kevin taps his fingers on the edge of his desk* Ah, the incident that led me to retire as a detective, huh? Well, it's a bit of a sensitive topic, but I'm open to discussing it with you. Shall we grab a coffee and talk about it in person?"


In [ ]:
test_eq(speaker.name, 'Kevin')

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()